In [21]:
import os
import sys
import requests
from datetime import datetime

import numpy as np
import pandas as pd
import yfinance as yf

In [22]:
file_name = '../data/price_1.csv'
start_date = '2022-01-01'
end_date = '2023-01-01'
# end_date = datetime.now().strftime('%Y-%m-%d')



def grab_price_data(start_date,end_date):
    #tickers_list = ['JPM', 'COST', 'IBM', 'HD', 'ARWR']
    tickers_list = ['GOOG','AMD','META']

# Store multiple result sets.
    
    full_price_history = []

    for ticker in tickers_list:
        price_history = yf.Ticker(ticker).history(period='max', start=start_date, end=end_date, interval='1d')

        for index, row in price_history.iterrows():
            row_data = row.to_dict()
            row_data['symbol'] = ticker
            row_data['datetime'] = index.strftime('%Y-%m-%d')  # Convert Pandas Timestamp to datetime string
            full_price_history.append(row_data)


    price_data = pd.DataFrame(full_price_history)
    
    
    price_data_ro = price_data
    price_data = price_data_ro[['datetime', 'symbol', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']] # rearrange column here
    price_data.to_csv('../data/inital_price_data.csv', index=False)
    price_data.head()


if os.path.exists('../data/inital_price_data.csv'):
    
    # Load the data
    price_data = pd.read_csv('../data/inital_price_data.csv')

else:

    # Grab the data and store it.
    grab_price_data(start_date, end_date)

    # Load the data
    price_data = pd.read_csv('../data/inital_price_data.csv')

# Display the head before moving on.
price_data.head()


def calculate_price_change():
    # Sort the data by symbol and datetime
    price_data.sort_values(by = ['symbol','datetime'], inplace = True)

    # calculate the change in price
    price_data['change_in_price'] = price_data['Close'].diff()

    return price_data.head()



def calculate_price_change2():
    mask = price_data['symbol'] != price_data['symbol'].shift(1)

    # For those rows, let's make the value null
    price_data['change_in_price'] = np.where(mask == True, np.nan, price_data['change_in_price'])

    # print the rows that have a null value, should only be 5
    return price_data[price_data.isna().any(axis = 1)]


def smoothed_df():
    # define the number of days out you want to predict
    days_out = 30

    # Group by symbol, then apply the rolling function and grab the Min and Max.
    price_data_smoothed = price_data.groupby(['symbol'])[['Close','Low','High','Open','Volume']].transform(lambda x: x.ewm(span = days_out).mean())

    # Join the smoothed columns with the symbol and datetime column from the old data frame.
    smoothed_df = pd.concat([price_data[['symbol','datetime']], price_data_smoothed], axis=1, sort=False)

    

    # create a new column that will house the flag, and for each group calculate the diff compared to 30 days ago. Then use Numpy to define the sign.
    smoothed_df['Signal_Flag'] = smoothed_df.groupby('symbol')['Close'].transform(lambda x : np.sign(x.diff(days_out)))

    # print the first 50 rows
    return smoothed_df.head(50)



def calculate_RSI():
    # Calculate the 14 day RSI
    n = 14

    # First make a copy of the data frame twice
    up_df, down_df = price_data[['symbol','change_in_price']].copy(), price_data[['symbol','change_in_price']].copy()

    # For up days, if the change is less than 0 set to 0.
    up_df.loc['change_in_price'] = up_df.loc[(up_df['change_in_price'] < 0), 'change_in_price'] = 0

    # For down days, if the change is greater than 0 set to 0.
    down_df.loc['change_in_price'] = down_df.loc[(down_df['change_in_price'] > 0), 'change_in_price'] = 0

    # We need change in price to be absolute.
    down_df['change_in_price'] = down_df['change_in_price'].abs()

    # Calculate the EWMA (Exponential Weighted Moving Average), meaning older values are given less weight compared to newer values.
    ewma_up = up_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())
    ewma_down = down_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())

    # Calculate the Relative Strength
    relative_strength = ewma_up / ewma_down

    # Calculate the Relative Strength Index
    relative_strength_index = 100.0 - (100.0 / (1.0 + relative_strength))

    # Add the info to the data frame.
    price_data['down_days'] = down_df['change_in_price']
    price_data['up_days'] = up_df['change_in_price']
    price_data['RSI'] = relative_strength_index

    # Display the head.
    return price_data.head(30)



def calculate_Stoc_Osc():
    # Calculate the Stochastic Oscillator
    n = 14

    # Make a copy of the high and low column.
    low_14, high_14 = price_data[['symbol','Low']].copy(), price_data[['symbol','High']].copy()

    # Group by symbol, then apply the rolling function and grab the Min and Max.
    low_14 = low_14.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = n).min())
    high_14 = high_14.groupby('symbol')['High'].transform(lambda x: x.rolling(window = n).max())

    # Calculate the Stochastic Oscillator.
    k_percent = 100 * ((price_data['Close'] - low_14) / (high_14 - low_14))

    # Add the info to the data frame.
    price_data['low_14'] = low_14
    price_data['high_14'] = high_14
    price_data['k_percent'] = k_percent

    # Display the head.
    return price_data.head(30)

def calculate_williams_R():
    # Calculate the Williams %R
    n = 14

    # Make a copy of the high and low column.
    low_14, high_14 = price_data[['symbol','Low']].copy(), price_data[['symbol','High']].copy()

    # Group by symbol, then apply the rolling function and grab the Min and Max.
    low_14 = low_14.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = n).min())
    high_14 = high_14.groupby('symbol')['High'].transform(lambda x: x.rolling(window = n).max())

    # Calculate William %R indicator.
    r_percent = ((high_14 - price_data['Close']) / (high_14 - low_14)) * - 100

    # Add the info to the data frame.
    price_data['r_percent'] = r_percent

    # Display the head.
    return price_data.head(30)


def calculate_MACD():
    # Calculate the MACD
    ema_26 = price_data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span = 26).mean())
    ema_12 = price_data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span = 12).mean())
    macd = ema_12 - ema_26

    # Calculate the EMA
    ema_9_macd = macd.ewm(span = 9).mean()

    # Store the data in the data frame.
    price_data['MACD'] = macd
    price_data['MACD_EMA'] = ema_9_macd

    # Print the head.
    return price_data.head(30)

def calculate_price_rate_of_change():
    # Calculate the Price Rate of Change
    n = 9

    # Calculate the Rate of Change in the Price, and store it in the Data Frame.
    price_data['Price_Rate_Of_Change'] = price_data.groupby('symbol')['Close'].transform(lambda x: x.pct_change(periods = n))

    # Print the first 30 rows
    return price_data.head(30)

def obv(group):
    
    # Grab the volume and close column.
    volume = group['Volume']
    change = group['Close'].diff()

    # intialize the previous OBV
    prev_obv = 0
    obv_values = []

    # calculate the On Balance Volume
    for i, j in zip(change, volume):

        if i > 0:
            current_obv = prev_obv + j
        elif i < 0:
            current_obv = prev_obv - j
        else:
            current_obv = prev_obv

        # OBV.append(current_OBV)
        prev_obv = current_obv
        obv_values.append(current_obv)
    
    # Return a panda series.
    return pd.Series(obv_values, index = group.index)
        
def calculate_obv():
# apply the function to each group
    obv_groups = price_data.groupby('symbol').apply(obv)

    # add to the data frame, but drop the old index, before adding it.
    price_data['On Balance Volume'] = obv_groups.reset_index(level=0, drop=True)

    # display the data frame.
    return price_data.head(30)

'''
    In this case, let's create an output column that will be 1 if the closing price at time 't' is greater than 't-1' and 0 otherwise.
    In other words, if the today's closing price is greater than yesterday's closing price it would be 1.
'''

# Create a column we wish to predict
def calculate_prediction():
    
# Group by the `Symbol` column, then grab the `Close` column.
    close_groups = price_data.groupby('symbol')['Close']

    # Apply the lambda function which will return -1.0 for down, 1.0 for up and 0.0 for no change.
    close_groups = close_groups.transform(lambda x : np.sign(x.diff()))

    # add the data to the main dataframe.
    price_data['Prediction'] = close_groups

    # for simplicity in later sections I'm going to make a change to our prediction column. To keep this as a binary classifier I'll change flat days and consider them up days.
    price_data.loc[price_data['Prediction'] == 0.0] = 1.0

    # print the head
    return price_data.head(50)
    # OPTIONAL CODE: Dump the data frame to a CSV file to examine the data yourself.
    # price_data.to_csv('final_metrics.csv')



#-----------------------------

calculate_price_change()
calculate_price_change2()
smoothed_df()

calculate_RSI()
calculate_Stoc_Osc()
calculate_williams_R()
calculate_MACD()
calculate_price_rate_of_change()
calculate_obv()
calculate_prediction()


,datetime,symbol,Open,High,Low,Close,Volume,Dividends,Stock Splits,change_in_price,...,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume,Prediction
319,2023-01-03,AMD,66.000000,66.879997,63.590000,64.019997,46851800.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.0,NaN
320,2023-01-04,AMD,65.000000,65.790001,63.310001,64.660004,47477100.0,0.0,0.0,0.640007,...,100.000000,NaN,NaN,NaN,NaN,0.014359,0.007977,NaN,47477100.0,1.0
321,2023-01-05,AMD,64.150002,64.349998,62.299999,62.330002,46159500.0,0.0,0.0,-2.330002,...,19.228261,NaN,NaN,NaN,NaN,-0.053957,-0.017406,NaN,1317600.0,-1.0
322,2023-01-06,AMD,63.150002,64.300003,60.049999,63.959999,70161300.0,0.0,0.0,1.629997,...,51.106267,NaN,NaN,NaN,NaN,-0.024855,-0.019929,NaN,71478900.0,1.0
323,2023-01-09,AMD,66.220001,69.320000,65.669998,67.239998,69741300.0,0.0,0.0,3.279999,...,74.486153,NaN,NaN,NaN,NaN,0.122000,0.022291,NaN,141220200.0,1.0
324,2023-01-10,AMD,66.669998,68.150002,66.559998,68.050003,41149600.0,0.0,0.0,0.810005,...,77.545660,NaN,NaN,NaN,NaN,0.242623,0.082013,NaN,182369800.0,1.0
325,2023-01-11,AMD,68.389999,69.129997,67.220001,69.059998,44470100.0,0.0,0.0,1.009995,...,80.849602,NaN,NaN,NaN,NaN,0.361555,0.152758,NaN,226839900.0,1.0
326,2023-01-12,AMD,70.070000,71.650002,67.180000,70.800003,70066200.0,0.0,0.0,1.740005,...,85.183335,NaN,NaN,NaN,NaN,0.518243,0.240591,NaN,296906100.0,1.0
327,2023-01-13,AMD,69.839996,71.099998,69.230003,71.000000,45757400.0,0.0,0.0,0.199997,...,85.615066,NaN,NaN,NaN,NaN,0.628785,0.330266,NaN,342663500.0,1.0
328,2023-01-17,AMD,70.870003,72.660004,70.650002,71.589996,42621300.0,0.0,0.0,0.589996,...,86.913065,NaN,NaN,NaN,NaN,0.728442,0.419480,0.118244,385284800.0,1.0


In [23]:
print('Before NaN Drop we have {} rows and {} columns'.format(price_data.shape[0], price_data.shape[1]))

# Any row that has a `NaN` value will be dropped.
price_data = price_data.dropna()

# Display how much we have left now.
print('After NaN Drop we have {} rows and {} columns'.format(price_data.shape[0], price_data.shape[1]))

# Print the head.
price_data.head()



Before NaN Drop we have 957 rows and 22 columns
After NaN Drop we have 918 rows and 22 columns


,datetime,symbol,Open,High,Low,Close,Volume,Dividends,Stock Splits,change_in_price,...,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume,Prediction
332,2023-01-23,AMD,72.220001,76.589996,71.540001,76.529999,84293200.0,0.0,0.0,6.459999,...,78.064019,60.049999,76.589996,99.637258,-0.362742,0.950718,0.611520,0.138162,430607000.0,1.0
333,2023-01-24,AMD,73.750000,75.639999,73.419998,74.699997,60822600.0,0.0,0.0,-1.830002,...,68.432342,60.049999,76.589996,88.573158,-11.426842,1.110242,0.714902,0.097722,369784400.0,-1.0
334,2023-01-25,AMD,72.900002,75.120003,72.089996,74.910004,49932600.0,0.0,0.0,0.210007,...,68.939783,60.049999,76.589996,89.842848,-10.157152,1.230070,0.820920,0.084709,419717000.0,1.0
335,2023-01-26,AMD,76.500000,77.080002,74.279999,75.160004,49583900.0,0.0,0.0,0.250000,...,69.610774,60.049999,77.080002,88.725791,-11.274209,1.321661,0.923375,0.061582,469300900.0,1.0
336,2023-01-27,AMD,73.699997,76.739998,73.489998,75.400002,58118600.0,0.0,0.0,0.239998,...,70.320969,65.669998,77.080002,85.276076,-14.723924,1.390713,1.018557,0.061972,527419500.0,1.0


In [24]:
price_data.to_csv(file_name)